In [1]:
import numpy as np
import pandas as pd
import os, shutil, re, csv,glob
# target folder name
sub_folder = ['d48','d77']
sheetname = ['averaged_before_led_df','dist_vel_acc_df']
RawExtract = pd.DataFrame()


In [ ]:
# delete previous folder, and create new blank folder to save extraction result
if os.path.exists(os.getcwd()+'/Raw_Extraction'):
    shutil.rmtree(os.getcwd()+'/Raw_Extraction')
os.makedirs(os.getcwd()+'/Raw_Extraction', mode = 0o777, exist_ok=False)

# STEP 1. raw data extraction

In [2]:
# 1. output of avg_before.xlsx
avg_bf_df = pd.DataFrame()
for fish_life in sub_folder:
    avg_before_path = os.getcwd()+ "/" + fish_life + "/dabest_plots/paired/averaged_before_led_df.csv"
    avg_bf_df = pd.read_csv(avg_before_path)
    avg_bf_df.to_csv(os.getcwd()+'/Raw_Extraction/%s_averaged_before_led.csv'%fish_life)

In [3]:
# 2. output of during_LED

# glob searching file name，例如 "*.txt" 或 "file_*.jpg"
file_pattern = "LED*"
during_led_df = pd.DataFrame()


for fish_life in sub_folder:
    print('rpocessing fish group:%s'%fish_life)
    during_sep_file = glob.glob(os.getcwd() + "/"+ fish_life + "/"+ file_pattern)
    #during_sep_file.append(os.getcwd()+ "/"+ fish_life + "/start_y_trajectory_ymag_yup")
    #during_sep_file.append(os.getcwd()+ "/"+ fish_life + "/trajectory_direction")
    loop=0
    for subf in during_sep_file:
        file_list = os.listdir(subf)
        #print(subf)
        
        for filename in file_list:
            if filename.endswith('.csv'):
                #print(filename)
                loop+=1
                csv_content = pd.read_csv(subf +"/"+filename)
                if loop == 1:
                    during_led_df = pd.concat( [during_led_df,csv_content], axis = 1)
                else:
                    column_idx = csv_content.columns.get_loc("uroa_control")
                    useful_content = csv_content.iloc[:, column_idx+1:]
                    during_led_df = pd.concat( [during_led_df,useful_content], axis = 1)
    
    
    # firstly sort by name, secondly sort by day
    during_led_df = during_led_df.sort_values(by= 'name')
    during_led_df = during_led_df.groupby('name').apply(lambda x: x.sort_values(by='day')).reset_index(drop=True)
    during_led_df.to_csv(os.getcwd()+'/Raw_Extraction/%s_during_led.csv'%fish_life,index = False)
    during_led_df = pd.DataFrame()

rpocessing fish group:d48
rpocessing fish group:d77


In [4]:
#3. output of diff by before to after

diff_df = pd.DataFrame()
for fish_life in sub_folder:
    during_led_total = pd.read_csv(os.getcwd()+'/Raw_Extraction/%s_during_led.csv'%fish_life)
    before_led_total = pd.read_csv(os.getcwd()+'/Raw_Extraction/%s_averaged_before_led.csv'%fish_life)
    diff_df = during_led_total.iloc[:,:3]
    dur_col_name =  during_led_total.columns[3:].tolist() #list
    print('during column name:%s'%dur_col_name)
    for colname in dur_col_name:
        if colname in before_led_total.columns:
            #print(colname)
            #during_col_data = during_led_total[name]
            #before_col_data = before_led_total[name]
            diff_col_data = during_led_total[colname] - before_led_total[colname]
            diff_df = pd.concat([diff_df,diff_col_data],axis = 1)
    diff_df.to_csv(os.getcwd()+'/Raw_Extraction/%s_diff_led.csv'%fish_life,index = False)
    dur_col_name = []
    diff_df = pd.DataFrame()

during column name:['dist', 'vel', 'acc', 'head_angle', 'y_mag', 'trajectory_angle', 'start_y_from_led', 'trajectory_angle.1', 'y_up']
during column name:['dist', 'vel', 'acc', 'head_angle', 'y_mag', 'trajectory_angle', 'start_y_from_led', 'trajectory_angle.1', 'y_up']


# STEP 2 calculation of z-score

In [5]:
# glob searching file name，例如 "*.txt" 或 "file_*.jpg"
file_pattern = "*diff*"
diff_led = pd.DataFrame()
z_score = pd.DataFrame()
diff_file_path = glob.glob(os.getcwd() + "/Raw_Extraction/"+file_pattern)
for i in range(len(diff_file_path)):
    
    print('Z_score calculation from:%s'%diff_file_path[i])
    diff_file = pd.read_csv(diff_file_path[i])
    z_score = diff_file.iloc[:,:3]
    colname = diff_file.columns
    #print(colname)
    for j in range (diff_file.shape[1] - 3):
        data = diff_file.iloc[:, j+3]
        average = data.mean()
        std_deviation = data.std()
        print(average,std_deviation)
        z_score['z_%s'%colname[j+3]] = (diff_file.iloc[:,j+3]-average)/std_deviation
    z_score.to_csv(os.getcwd()+'/Raw_Extraction/%s_z_score.csv'%sub_folder[i],index = False)

Z_score calculation from:E:\RA\KILLIFISH/Raw_Extraction\d48_diff_led.csv
59.496719061348266 257.06641849349097
0.6327157509737876 2.8358457849114607
18.981471501205846 85.07536902275015
-26.961693934587323 77.8576765054616
38.7768654441461 81.81440957283627
-10.879703101404901 96.84505227659187
-2.5052251680768975 62.1774007021324
12.808449998489072 39.03195779916418
Z_score calculation from:E:\RA\KILLIFISH/Raw_Extraction\d77_diff_led.csv
71.64177437138189 222.03986692984202
0.7697075488677766 2.454105895501354
23.091225316678045 73.6231732794538
-47.31164484518505 94.49126476413662
49.537210914747796 73.88559193121638
-19.055673937100075 79.05207576402509
1.2932728044294994 64.21831803426748
18.487639161984525 36.5311716753007


# STEP 3 calculation of z diff

In [20]:
file_pattern = "*z_score*"
z_score_ori = pd.DataFrame()
df_move_col = pd.DataFrame()
zscore_file_path = glob.glob(os.getcwd() + "/Raw_Extraction/"+file_pattern)
sub_idx=0
for path in zscore_file_path:
    print(path)
    merge_group_z = pd.DataFrame()
    #df_move_col = pd.DataFrame()
    z_score_ori = pd.read_csv(path)
    name_grouped = z_score_ori.groupby('name')
    for name, group in name_grouped:
        #print(name,group)
        # dealing with day data missing issue
        
        if group.shape[0] == 4:
            diffD1D2 = group.iloc[0,3:]-group.iloc[1,3:]
            diffD1D3 = group.iloc[0,3:]-group.iloc[2,3:]
            diffD2D3 = group.iloc[1,3:]-group.iloc[2,3:]
            diffD1D4 = group.iloc[0,3:]-group.iloc[3,3:]
            diffD2D4 = group.iloc[1,3:]-group.iloc[3,3:]
        
        # 待修正，可能是123，134，234天的数据，做分类计算
        elif group.shape[0] == 3:
            
            if 1 not in group['day'].tolist(): #D2D3,D2D4
                #print("D2D3,D2D4:\n",group['day'])
                diffD2D3 = group.iloc[0,3:]-group.iloc[1,3:]
                diffD2D4 = group.iloc[0,3:]-group.iloc[2,3:]
                diffD1D2 = diffD1D3 = diffD1D4 = group.iloc[0,3:]-group.iloc[0,3:]
            elif 2 not in group['day'].tolist(): #D1D3 D1D4
                #print("D1D3 D1D4:\n",group['day'])
                diffD1D3 = group.iloc[0,3:]-group.iloc[1,3:]
                diffD1D4 = group.iloc[0,3:]-group.iloc[2,3:]
                diffD1D2 = diffD2D3 = diffD2D4 = group.iloc[0,3:]-group.iloc[0,3:]
            elif 3 not in group['day'].tolist(): #D1D2,D1D4,D2D4
                #print("D1D2,D1D4,D2D4:\n",group['day'])
                diffD1D2 = group.iloc[0,3:]-group.iloc[1,3:]
                diffD1D4 = group.iloc[0,3:]-group.iloc[2,3:]
                diffD2D4 = group.iloc[1,3:]-group.iloc[2,3:]
                diffD1D3 = diffD2D3 = group.iloc[0,3:]-group.iloc[0,3:]
            elif 4 not in group['day'].tolist():#D1D2,D1D3,D2D3
                #print("D1D2,D1D3,D2D3:\n",group['day'])
                diffD1D2 = group.iloc[0,3:]-group.iloc[1,3:]
                diffD1D3 = group.iloc[0,3:]-group.iloc[2,3:]
                diffD2D3 = group.iloc[1,3:]-group.iloc[2,3:]
                diffD2D4 = diffD1D4 = group.iloc[0,3:]-group.iloc[0,3:]

        # 只有两天的数据，分类计算 12，13，14，23，24，34
        elif group.shape[0] == 2: 
            if 1 and 2 in group['day'].tolist():
                #print('day1 and 2:\n',group['day'])
                diffD2D4 = diffD1D4 = diffD2D3 = diffD1D3 = group.iloc[0,3:]-group.iloc[0,3:]
                diffD1D2 = group.iloc[0,3:]-group.iloc[1,3:]
            elif 1 and 3 in group['day'].tolist():
                #print('day1 and 3:\n',group['day'])
                diffD2D4 = diffD1D4 = diffD2D3 = diffD1D2 = group.iloc[0,3:]-group.iloc[0,3:]
                diffD1D3 = group.iloc[0,3:]-group.iloc[1,3:]
            elif 1 and 4 in group['day'].tolist():
                #print('day1 and 3:\n',group['day'])
                diffD2D4 = diffD1D3 = diffD2D3 = diffD1D2 = group.iloc[0,3:]-group.iloc[0,3:]
                diffD1D4 = group.iloc[0,3:]-group.iloc[1,3:]
                
            elif 2 and 3 in group['day'].tolist():
                #print('day1 and 3:\n',group['day'])
                diffD2D4 = diffD1D4 = diffD1D3 = diffD1D2 = group.iloc[0,3:]-group.iloc[0,3:]
                diffD2D3 = group.iloc[0,3:]-group.iloc[1,3:]            
            elif 2 and 4 in group['day'].tolist():
                #print('day1 and 3:\n',group['day'])
                diffD1D3 = diffD1D4 = diffD2D3 = diffD1D2 = group.iloc[0,3:]-group.iloc[0,3:]
                diffD2D4 = group.iloc[0,3:]-group.iloc[1,3:]            
            else:
                diffD2D4 = diffD1D4 = diffD2D3 = diffD1D3 = diffD1D2 = group.iloc[0,3:]-group.iloc[0,3:]
        else:
            diffD2D4 = diffD1D4 = diffD2D3 = diffD1D3 = diffD1D2 = group.iloc[0,3:]-group.iloc[0,3:]
        
        one_group_diffZ = pd.concat([diffD1D2, diffD1D3,diffD1D4, diffD2D3, diffD2D4], axis=0)
        one_group_diffZ = pd.DataFrame(one_group_diffZ)
        one_group_diffZ = one_group_diffZ.transpose()
        one_group_diffZ['name'] = name
        one_group_diffZ = one_group_diffZ.set_index('name') 
        merge_group_z = pd.concat([merge_group_z,one_group_diffZ],axis = 0)
    
    merge_group_z = merge_group_z.abs()
    
    # rename columns by adding D1D2 etc.
    new_columns = []
    for i, col in enumerate(merge_group_z.columns):
        if i < 8:
            new_columns.append("D1D2" + col)
        elif i>=8 and i< 16:
            new_columns.append("D1D3" + col)
        elif i>=16 and i< 24:
            new_columns.append("D1D4" + col)
        elif i>=24 and i< 32:
            new_columns.append("D2D3" + col)
        elif i>=32:
            new_columns.append("D2D4" + col)
    merge_group_z.columns = new_columns
    
    
    # remove columns to get a new df
    df_move_col = pd.DataFrame(index = merge_group_z.index)
    for i in range(int(merge_group_z.shape[1]/5)):
        k=0
        for j in range(int(merge_group_z.shape[1]/8)):
        # 0,8,16,24,32
            df_move_col = pd.concat([df_move_col,merge_group_z.iloc[:,k+i]],axis = 1)
            k+=8
    
    df_move_col.to_csv(os.getcwd()+'/Raw_Extraction/%s_z_by_day.csv'%sub_folder[sub_idx],index = True)
    sub_idx += 1
    

E:\RA\KILLIFISH/Raw_Extraction\d48_z_score.csv
D1D3 D1D4:
 20    1
21    3
22    4
Name: day, dtype: int64
D1D2,D1D4,D2D4:
 147    1
148    2
149    4
Name: day, dtype: int64
day1 and 2:
 222    1
223    2
Name: day, dtype: int64
E:\RA\KILLIFISH/Raw_Extraction\d77_z_score.csv
D1D2,D1D3,D2D3:
 40    1
41    2
42    3
Name: day, dtype: int64
D2D3,D2D4:
 51    2
52    3
53    4
Name: day, dtype: int64
D2D3,D2D4:
 126    2
127    3
128    4
Name: day, dtype: int64
D1D3 D1D4:
 177    1
178    3
179    4
Name: day, dtype: int64
D1D2,D1D4,D2D4:
 204    1
205    2
206    4
Name: day, dtype: int64


pandas.core.frame.DataFrame

   D1A  D1B  D1C  D1D  D1E  D2F  D2G  D2H  D2I  D2J
0    1    4    7   10   13   16   19   22   25   28
1    2    5    8   11   14   17   20   23   26   29
2    3    6    9   12   15   18   21   24   27   30
